In [41]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from time import time

np.random.seed(1337)
df_train = pd.read_csv('train_with_age.csv') # this is going to be our training data.
df_test = pd.read_csv('train.csv') # select rows without missing ages as testing data.

In [42]:
df_train.shape
df_test.shape

(891, 12)

In [43]:
df_test.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [44]:
# Name, Ticket, Cabin may not be as useful as other columns.
df_train = df_train.drop(['Name','Ticket', 'Cabin'], axis=1)
df_test = df_test.drop(['Name','Ticket', 'Cabin'], axis=1)

In [45]:
# remove rows with empty ages in test data.
df_test = df_test.dropna()

In [46]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Sex            891 non-null object
Age            891 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Fare           891 non-null float64
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(2)
memory usage: 62.7+ KB


In [47]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 0 to 890
Data columns (total 9 columns):
PassengerId    712 non-null int64
Survived       712 non-null int64
Pclass         712 non-null int64
Sex            712 non-null object
Age            712 non-null float64
SibSp          712 non-null int64
Parch          712 non-null int64
Fare           712 non-null float64
Embarked       712 non-null object
dtypes: float64(2), int64(5), object(2)
memory usage: 55.6+ KB


In [48]:
# Map male to 1 and female to 0.
df_train['Sex'] = df_train['Sex'].map({'female':0, 'male':1})
df_test['Sex'] = df_test['Sex'].map({'female':0, 'male':1})

In [49]:
scaler = StandardScaler()
features = ['Pclass','Sex','Age','Fare']

X_train = scaler.fit_transform(df_train[features].values)
y_train = df_train['Survived'].values
# For categorical variables where no such ordinal relationship exists, the integer encoding is not enough.
# Don't wanna confuse machine learning algorithm.
y_train_onehot = pd.get_dummies(df_train['Survived']).values 

X_test = scaler.transform(df_test[features].values)
y_test = df_test['Survived'].values

In [50]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=0, verbose=3)
model = model.fit(X_train, df_train['Survived'].values)
y_prediction = model.predict(X_test)
accuracy = np.sum(y_prediction == y_test) / float(len(y_test))
print(accuracy)

building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10
0.966292134831


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished


In [51]:
from keras.models import Sequential
from keras.layers import Dense, Activation

start = time()

model = Sequential()
model.add(Dense(input_dim=4, output_dim=2))
model.add(Activation("softmax"))
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

model.fit(X_train, y_train_onehot)

/Users/lingxiwu/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=4, units=2)`
  import sys


Epoch 1/10
891/891 [==============================] - 0s - loss: 1.4459 - acc: 0.3345     
Epoch 2/10
891/891 [==============================] - ETA: 0s - loss: 1.0238 - acc: 0.468 - 0s - loss: 1.2121 - acc: 0.3288     
Epoch 3/10
891/891 [==============================] - 0s - loss: 1.0275 - acc: 0.3345     
Epoch 4/10
891/891 [==============================] - 0s - loss: 0.8894 - acc: 0.3333     
Epoch 5/10
891/891 [==============================] - 0s - loss: 0.7891 - acc: 0.4029     
Epoch 6/10
891/891 [==============================] - 0s - loss: 0.7171 - acc: 0.5948     
Epoch 7/10
891/891 [==============================] - 0s - loss: 0.6652 - acc: 0.6442     
Epoch 8/10
891/891 [==============================] - 0s - loss: 0.6279 - acc: 0.6779     
Epoch 9/10
891/891 [==============================] - 0s - loss: 0.6001 - acc: 0.6970     
Epoch 10/10
891/891 [==============================] - 0s - loss: 0.5792 - acc: 0.7318     


In [55]:
y_prediction = model.predict_classes(X_test)
score = np.sum(y_prediction == y_test)/float(len(y_test))
print("\nPrediction Accuracy: %5.3f." % score )

 32/712 [>.............................] - ETA: 0s
Prediction Accuracy: 0.725.
